In [1]:
%reload_ext autoreload
%autoreload 2

# Imports

In [2]:
from kret_notebook import *  # NOTE import first
from kret_lgbm._core.lgbm_nb_imports import *
from kret_lightning._core.lightning_nb_imports import *
from kret_matplotlib._core.mpl_nb_imports import *
from kret_np_pd._core.np_pd_nb_imports import *
from kret_optuna._core.optuna_nb_imports import *
from kret_polars._core.polars_nb_imports import *
from kret_rosetta._core.rosetta_nb_imports import *
from kret_sklearn._core.sklearn_nb_imports import *
from kret_torch_utils._core.torch_nb_imports import *
from kret_tqdm._core.tqdm_nb_imports import *
from kret_type_hints._core.types_nb_imports import *
from kret_utils._core.utils_nb_imports import *

# from kret_wandb._core.wandb_nb_imports import *  # NOTE this is slow to import

[kret_np_pd._core.np_pd_nb_imports] Imported kret_np_pd._core.np_pd_nb_imports in 0.0275 seconds
Loaded environment variables from /Users/Akseldkw/coding/kretsinger/.env
[kret_lgbm._core.lgbm_nb_imports] Imported kret_lgbm._core.lgbm_nb_imports in 0.0010 seconds
[kret_lightning._core.lightning_nb_imports] Imported kret_lightning._core.lightning_nb_imports in 0.0077 seconds
[kret_matplotlib._core.mpl_nb_imports] Imported kret_matplotlib._core.mpl_nb_imports in 0.0920 seconds
[kret_optuna._core.optuna_nb_imports] Imported kret_optuna._core.optuna_nb_imports in 0.0003 seconds
[kret_polars._core.polars_nb_imports] Imported kret_polars._core.polars_nb_imports in 0.0988 seconds
[kret_rosetta._core.rosetta_nb_imports] Imported kret_rosetta._core.rosetta_nb_imports in 0.0000 seconds
[kret_sklearn._core.sklearn_nb_imports] Imported kret_sklearn._core.sklearn_nb_imports in 0.1511 seconds
[kret_torch_utils._core.torch_nb_imports] Imported kret_torch_utils._core.torch_nb_imports in 0.5295 seconds


In [3]:
from kret_lightning.examples.cifar10_module import *
from kret_lightning.examples.cifar10_datamodule import *

# Load Data

In [4]:
cifar_dm = CIFAR10DataModule(UKS_CONSTANTS.DATA_DIR / "CIFAR10", batch_size=64, num_workers=8)

In [5]:
cifar_dm.data_dir

PosixPath('/Users/Akseldkw/coding/data_kretsinger/CIFAR10')

# Implementation

In [6]:
static_args = TrainerStaticDefaults.OPTUNA_FINE_TUNE.copy()
# static_args["max_epochs"] = 10
static_args

{'max_epochs': 50,
 'limit_train_batches': None,
 'limit_val_batches': None,
 'log_every_n_steps': 20,
 'enable_model_summary': False,
 'enable_checkpointing': False,
 'max_time': {'hours': 2}}

In [ ]:
def objective(trial: optuna.trial.Trial) -> float:
    # import traceback

    # try:

    # preset = trial.suggest_categorical("preset", ["tiny", "small", "medium", "large", "xlarge"])
    num_blocks = trial.suggest_int("num_blocks", 2, 3)  # 1=tiny, 2=small, 3=medium, 4=large
    num_filters = trial.suggest_int("num_filters", 50, 150, log=True)
    dropout_rate = trial.suggest_float("dropout_rate", 0.28, 0.35)
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
    l1 = trial.suggest_float("l1", 1e-4, 1e-3, log=True)
    l2 = trial.suggest_float("l2", 1e-5, 1e-3, log=True)

    model = CIFAR10ResNet(
        num_blocks=num_blocks,
        num_filters=num_filters,
        dropout_rate=dropout_rate,
        lr=lr,
        l1_penalty=l1,
        l2_penalty=l2,
        patience=8,
    )
    model._sweep_mode = True  # NOTE important!
    dynamic_args = TrainerDynamicDefaults.trainer_dynamic_defaults(model, cifar_dm, logtype=None, trial=trial)
    trainer_args = static_args | dynamic_args

    trainer = L.Trainer(**trainer_args)  # New trainer per trial!
    assert trainer.logger is not None
    trainer.logger.log_hyperparams(model.hparams_initial)
    trainer.fit(model, datamodule=cifar_dm, **TrainerStaticDefaults.TRAINER_FIT)

    return trainer.callback_metrics["val_f1"].item()


# except Exception as e:
#     print(f"\n{'='*80}")
#     print(f"Exception in trial {trial.number}:")
#     print(f"{'='*80}")
#     traceback.print_exc()
#     print(f"{'='*80}\n")
#     raise  # Re-raise so Optuna knows the trial failed

In [8]:
OptunaDefaults.CREATE_STUDY_DEFAULTS

{'pruner': <optuna.pruners._hyperband.HyperbandPruner at 0x17f09a0c0>,
 'load_if_exists': True,
 'storage': 'sqlite://///Users/Akseldkw/coding/data_kretsinger/optuna_studies/optuna_studies.db'}

In [ ]:
study = optuna.create_study(
    study_name="cifar10_resnet_f1_finetune",
    direction="maximize",  # Maximize F1 score (higher is better)
    **OptunaDefaults.CREATE_STUDY_DEFAULTS,
)

[I 2026-01-22 23:21:26,382] A new study created in RDB with name: cifar10_resnet_f1_finetune


In [10]:
optim_args = OptunaDefaults.study_n_hours(3) | OptunaDefaults.OPTIM_STUDY_DEF
# optim_args["n_jobs"] = 1
optim_args

{'n_trials': None,
 'timeout': 10800,
 'n_jobs': 1,
 'gc_after_trial': True,
 'show_progress_bar': True}

In [ ]:
study.optimize(objective, **optim_args)

   0%|          | 00:00/3:00:00

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores


Saving hparams, ignoring ()
Saving hparams, ignoring ()


Loading `train_dataloader` to estimate number of stepping batches.


Output()

...

Metric val_loss improved by 0.087 >= min_delta = 0.0001. New best score: 1.493


Metric val_loss improved by 0.296 >= min_delta = 0.0001. New best score: 1.197


Metric val_loss improved by 0.197 >= min_delta = 0.0001. New best score: 1.000


Metric val_loss improved by 0.008 >= min_delta = 0.0001. New best score: 0.992


Metric val_loss improved by 0.152 >= min_delta = 0.0001. New best score: 0.839


# Evaluate

## Load

In [ ]:
study_names = optuna.study.get_all_study_names(storage=UKS_CONSTANTS.OPTUNA_STORAGE_DB)
study_names

['cifar10_resnet', 'cifar10_resnet_v1', 'cifar10_resnet_v2']

In [ ]:
study_summaries = optuna.study.get_all_study_summaries(storage=UKS_CONSTANTS.OPTUNA_STORAGE_DB)

In [ ]:
studyload = study_summaries[0]

In [ ]:
studyload.n_trials

77

In [ ]:
studyload.datetime_start

datetime.datetime(2026, 1, 19, 11, 4, 24, 454090)

In [ ]:
study.best_trial.params

{'num_blocks': 2,
 'num_filters': 128,
 'dropout_rate': 0.32117446313532205,
 'lr': 0.0003469745658031787,
 'l1': 5.1683361683017804e-05,
 'l2': 8.742339664966662e-05}

In [ ]:
out = optuna.importance.get_param_importances(study)

In [ ]:
out

{'num_filters': np.float64(0.3417231459629884),
 'l2': np.float64(0.253645835286912),
 'dropout_rate': np.float64(0.20035574159074557),
 'lr': np.float64(0.13402967229963758),
 'l1': np.float64(0.04068302844741069),
 'num_blocks': np.float64(0.029562576412305672)}

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
# Plot parameter importances
optuna.visualization.plot_param_importances(study)

In [ ]:
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
UKS_TH_UTILS.func_to_typed_dict(optuna.visualization.plot_optimization_history)

from typing import TypedDict

class Plot_optimization_history_TypedDict(TypedDict, total=False):
    study: 'Study | Sequence[Study]'
    target: 'Callable[[FrozenTrial], float] | None'  # = None
    target_name: 'str'  # = 'Objective Value'
    error_bar: 'bool'  # = False


In [ ]:
study.n_trials

AttributeError: 'Study' object has no attribute 'n_trials'